In [1]:
using LinearAlgebra
using Plots
using DifferentialEquations
using Roots
using OrdinaryDiffEq
using BoundaryValueDiffEq

In [10]:
# Problem parameters
μₑ = 398600 # [km^3/s^2]
μₘ = 4903
R = 384400  # Earth-Moon distance [km]

R_Earth = 6378 # Earth radius [km]
R_Moon = 1738  # Moon radius  [km]
h = 200

μ₁ = μₑ
μ₂ = μₘ

# Synodic reference frame parameters
R₁ = μ₂/(μ₂+μ₁)*R        # Earth-CM distance
R₂ = μ₁/(μ₂+μ₁)*R        # Moon-CM distance
r1 = [-R₁; 0; 0]
r2 = [ R₂; 0; 0]
ωₛ = sqrt((μ₁ + μ₂)/(R^3))

params = (μ₁,μ₂,r1,r2,ωₛ)

(398600, 4903, [-4670.877787773573, 0.0, 0.0], [379729.1222122264, 0.0, 0.0], 2.665313601093969e-6)

In [11]:
# Initial guess
v_inj = 10.92367104     # [km/s]
ϕ_inj = deg2rad(47.70061087)

# Initial conditions (synodic frame)
r0 = [-cos(ϕ_inj)*(h+R_Earth)-R₁; -sin(ϕ_inj)*(h+R_Earth); 0] # No z-position/velocity
v0 = [v_inj*sin(ϕ_inj); -v_inj*cos(ϕ_inj); 0]
x0 = [r0; v0]

6-element Array{Float64,1}:
 -9097.902229126235
 -4865.340542620438
     0.0
     8.07956515432666
    -7.351681161960514
     0.0

In [12]:
# 1a) Find L₁ point
f₁(x) = -μ₁*(x-R₂)^2 + μ₂*(x+R₁)^2 + ωₛ^2*abs(x)*(x+R₁)^2*(x-R₂)^2
x₁ = find_zero(f₁, .5*R)
L₁ = [x₁; 0; 0]

3-element Array{Float64,1}:
 321709.22627414274
      0.0
      0.0

In [39]:
# 1b) Use a fixed-time, single-shoot method to hit L₁
t_final = 3*24*3600

# Let's play around with BVPs and Shooting Methods in DifferentialEquations
function CR3BP_EOM(x, t, dx)
    ω_vec = [0;0;1]*ωₛ
    r3 = x[1:3]
    velocity = x[4:6]

    r13 = r3-r1
    r23 = r3-r2

    acceleration = -μ₁*r13/norm(r13)^3 -μ₂*r23/norm(r23)^3 -    # gravity
                    cross(ω_vec, cross(ω_vec, r3)) -            # centripetal
                    2*cross(ω_vec, velocity)                    # coriolis

    dx[1:3] = velocity
    dx[4:6] = acceleration
end

function bc_L1(residual, u)
    residual[1] = u[end][1] - L₁[1]
    residual[2] = u[end][2] - L₁[2]
    residual[3] = u[end][3] - L₁[3]
end



bc_L1 (generic function with 1 method)

In [40]:
bvp = BVProblem(CR3BP_EOM, bc_L1, x0, [0;t_final])

MethodError: MethodError: objects of type Array{Float64,1} are not callable
Use square brackets [] for indexing an Array.